In [ ]:
##################################################################
########### FINAL SCRIPT TO CONVERT CORA TO GIS FILE #############
##################################################################

import numpy as np
import s3fs
import xarray as xr
from scipy.interpolate import griddata
from tqdm import tqdm

# Tampa bounding box (can adjust for a user-specified ROI)
LON_MIN, LON_MAX = -82.7, -82.2
LAT_MIN, LAT_MAX = 27.7, 28.2

# Regular grid over Tampa
lon_grid = np.linspace(LON_MIN, LON_MAX, 500) #grid size is 500 - fairly fine resolution but can make coarser/finer depending on runtime!
lat_grid = np.linspace(LAT_MIN, LAT_MAX, 500)
lon2d, lat2d = np.meshgrid(lon_grid, lat_grid)

# CORA hourly surge file (change the year at the end if the time frame is different)
S3_PATH = "noaa-nos-cora-pds/V1.1/assimilated/native_grid/fort.63_2017.nc"

# Time window: Sept 9–12, 2017 (change for specific use-case)
START = "2017-09-10T00:00:00"
END   = "2017-09-11T23:00:00"

# Load CORA hourly surge from S3
fs = s3fs.S3FileSystem(anon=True)
with fs.open(S3_PATH, 'rb') as file:
    with xr.open_dataset(file) as ds_raw:
        ds = ds_raw.sel(time=slice(START, END))
        x = ds["x"].values
        y = ds["y"].values
        times = ds["time"].values

        # Build 3D surface cube: [time, lat, lon]
        surface_stack = []

        for t in tqdm(times, desc="Interpolating hourly surge"):
            zeta = ds["zeta"].sel(time=t).compute()
            surge_interp = griddata(
                (x, y),
                zeta.values,
                (lon2d, lat2d),
                method="nearest"
            )
            surface_stack.append(surge_interp)

        # Package into DataArray
        da_surface = xr.DataArray(
            np.array(surface_stack),
            coords={
                "time": times,
                "lat": lat_grid,
                "lon": lon_grid
            },
            dims=["time", "lat", "lon"],
            name="surge_elevation"
        )

        # Metadata for ArcGIS Pro
        da_surface.attrs.update({
            "units": "meters",
            "long_name": "Surge elevation",
            "_FillValue": -9999.0
        })
        da_surface["lat"].attrs.update({"units": "degrees_north", "_FillValue": -9999.0})
        da_surface["lon"].attrs.update({"units": "degrees_east", "_FillValue": -9999.0})
        da_surface["time"].attrs.pop("units", None)
        da_surface["time"].encoding["units"] = "hours since 2017-09-09 00:00:00"

        # Export NetCDF
        ds_out = xr.Dataset({"surge_elevation": da_surface})
        ds_out.attrs["title"] = "Hurricane Irma Surge Surface - Tampa"
        ds_out.attrs["description"] = "Hourly surge elevation over Tampa Bay during Hurricane Irma (Sept 9–12, 2017)"

        ds_out.to_netcdf("C:/Users/wyn14279/Documents/irma_surge_surface2.nc")
        print("✅ Saved: irma_surge_surface2.nc (Multidimensional Raster–ready)")

Interpolating hourly surge: 100%|██████████| 48/48 [12:43<00:00, 15.91s/it]

✅ Saved: irma_surge_surface2.nc (Multidimensional Raster–ready)
